In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


In [3]:
df = pd.read_csv('../data/Telco-Customer-Churn.csv')

In [4]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
df.shape

(7043, 21)

1 TotalCharges delete empty rows
    #Replacing spaces with null values in total charges column
    #Dropping null values from total charges column which contain .15% missing data 
2 Convert TotalCharges to float
3 replace 'No internet service' to No for 6 columns 
4 Senior Citizen transform 1 and 0 to Yes and No
5 Transform Tenure column to categorical variable
6 #Separating catagorical and numerical columns

7 split different columns per type of operation (id, target, numerical, cat_bin, cat_multi)
8 LabelEncode binary columns
9 get_dummies for multi columns
10 Scaling numerical columns
11 dropping original values merging scaled values for numerical columns

In [6]:
#Replacing spaces with null values in total charges column
df['TotalCharges'] = df["TotalCharges"].replace(" ",np.nan)

In [7]:
# Dropping null values from total charges column
df = df[df["TotalCharges"].notnull()]
df = df.reset_index()[df.columns]

In [11]:
# 2 Convert TotalCharges to float
df["TotalCharges"] = df["TotalCharges"].astype(float)

In [17]:
# 3 replace 'No internet service' to No for 6 columns 
replace_cols = [ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport','StreamingTV', 'StreamingMovies']

for i in replace_cols : 
    df[i]  = df[i].replace({'No internet service' : 'No'})


In [19]:
# 4 Senior Citizen transform 1 and 0 to Yes and No
df["SeniorCitizen"] = df["SeniorCitizen"].replace({1:"Yes",0:"No"})

In [20]:
# 5 Transform Tenure column to categorical variable

def tenure_lab(df) :
    
    if df["tenure"] <= 12 :
        return "Tenure_0-12"
    elif (df["tenure"] > 12) & (df["tenure"] <= 24 ):
        return "Tenure_12-24"
    elif (df["tenure"] > 24) & (df["tenure"] <= 48) :
        return "Tenure_24-48"
    elif (df["tenure"] > 48) & (df["tenure"] <= 60) :
        return "Tenure_48-60"
    elif df["tenure"] > 60 :
        return "Tenure_gt_60"
    
df["tenure_group"] = df.apply(lambda df:tenure_lab(df), axis = 1)


In [22]:
# 6 Separating catagorical and numerical columns
Id_column     = ['customerID']
target_column = ["Churn"]
cat_columns   = df.nunique()[df.nunique() < 6].keys().tolist()
cat_columns   = [x for x in cat_columns if x not in target_column]
num_columns   = [x for x in df.columns if x not in cat_columns + target_column + Id_column]

In [23]:
# 7 split different columns per type of operation (cat_bin, cat_multi)
binary_columns   = df.nunique()[df.nunique() == 2].keys().tolist()
# multi columns = more than 2 values
multi_columns = [i for i in cat_columns if i not in binary_columns]


In [27]:
# 8 LabelEncode binary columns
le = LabelEncoder()
for i in binary_columns :
    df[i] = le.fit_transform(df[i])


In [28]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_group
0,7590-VHVEG,0,0,1,0,1,0,No phone service,DSL,0,...,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0,Tenure_0-12
1,5575-GNVDE,1,0,0,0,34,1,No,DSL,1,...,0,0,0,One year,0,Mailed check,56.95,1889.50,0,Tenure_24-48
2,3668-QPYBK,1,0,0,0,2,1,No,DSL,1,...,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1,Tenure_0-12
3,7795-CFOCW,1,0,0,0,45,0,No phone service,DSL,1,...,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0,Tenure_24-48
4,9237-HQITU,0,0,0,0,2,1,No,Fiber optic,0,...,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1,Tenure_0-12


In [29]:
# 9 get_dummies for multi columns
df = pd.get_dummies(data = df,columns = multi_columns )

In [31]:
# 10 Scaling numerical columns
std = StandardScaler()
scaled = std.fit_transform(df[num_columns])
scaled = pd.DataFrame(scaled,columns=num_columns)

In [33]:
# 11 dropping original values merging scaled values for numerical columns

df_df_og = df.copy()
df = df.drop(columns = num_columns,axis = 1)
df = df.merge(scaled,left_index=True,right_index=True,how = "left")


In [38]:
# 12 Save DataFrame
df.to_csv("../data/telco_churn_processed.csv", index=False)

In [39]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,...,PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_Tenure_0-12,tenure_group_Tenure_12-24,tenure_group_Tenure_24-48,tenure_group_Tenure_48-60,tenure_group_Tenure_gt_60,tenure,MonthlyCharges,TotalCharges
0,7590-VHVEG,0,0,1,0,0,0,1,0,0,...,1,0,1,0,0,0,0,-1.280248,-1.161694,-0.994194
1,5575-GNVDE,1,0,0,0,1,1,0,1,0,...,0,1,0,0,1,0,0,0.064303,-0.260878,-0.173740
2,3668-QPYBK,1,0,0,0,1,1,1,0,0,...,0,1,1,0,0,0,0,-1.239504,-0.363923,-0.959649
3,7795-CFOCW,1,0,0,0,0,1,0,1,1,...,0,0,0,0,1,0,0,0.512486,-0.747850,-0.195248
4,9237-HQITU,0,0,0,0,1,0,0,0,0,...,1,0,1,0,0,0,0,-1.239504,0.196178,-0.940457


In [40]:
df_prvi = pd.read_csv('../data/telco_churn_processed.csv')

In [41]:
df_prvi.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,...,PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_Tenure_0-12,tenure_group_Tenure_12-24,tenure_group_Tenure_24-48,tenure_group_Tenure_48-60,tenure_group_Tenure_gt_60,tenure,MonthlyCharges,TotalCharges
0,7590-VHVEG,0,0,1,0,0,0,1,0,0,...,1,0,1,0,0,0,0,-1.280248,-1.161694,-0.994194
1,5575-GNVDE,1,0,0,0,1,1,0,1,0,...,0,1,0,0,1,0,0,0.064303,-0.260878,-0.173740
2,3668-QPYBK,1,0,0,0,1,1,1,0,0,...,0,1,1,0,0,0,0,-1.239504,-0.363923,-0.959649
3,7795-CFOCW,1,0,0,0,0,1,0,1,1,...,0,0,0,0,1,0,0,0.512486,-0.747850,-0.195248
4,9237-HQITU,0,0,0,0,1,0,0,0,0,...,1,0,1,0,0,0,0,-1.239504,0.196178,-0.940457
